# Stitching multiple images

This notebook is used to find a solution to the multi-graph syncronization problem and stitch images together.

In [1]:
import cv2 as cv
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import ipynb.fs.defs.Utils as Utils
import ipynb.fs.defs.ImageStitcher as ImageStitcher

In [6]:
def multi_graph_stitching(dataset_name,
                            imgs,
                            T_norm,
                            M, 
                            C,
                            idx_ref = 0,
                            idxs = None,
                            verbose = True,
                            save_output = True,
                            beautify = True,
                            stitching_dir = "stitched",
                            graph_stitching_dir = "multi_graph_stitching",
                            warp_shape = [10000,10000] ):
    
    output_dir = os.path.join(os.path.join(stitching_dir,dataset_name),graph_stitching_dir)
    if save_output:
        if os.path.isdir(output_dir):   
            shutil.rmtree(output_dir)
        if not os.path.isdir(output_dir):   
            os.makedirs(output_dir)
          

    S = (np.eye(M.shape[0])-C@np.linalg.pinv(C))@M.transpose()@M
    w,v = np.linalg.eig(S)
    
    w = np.real(w)
    v = np.real(v)
    d = 3
    if C.size == 0:
        off = 0
    else:
        off = np.linalg.matrix_rank(C)
        
    indx = np.argsort(w)
    indxs = np.array([indx]*indx.shape[0])
    
    w_sort = w[indx]
    v_sort = np.take_along_axis(v, indxs, axis=1)
    
    u_hat = v_sort[:,off:d+off]
    
    U = Utils.split_states(u_hat)[:len(imgs)] #Get the state of each node

    H = Utils.get_homographies_from_states(U, idx_ref)
    
    stitcher = ImageStitcher.ImageStitcher(imgs)

    H, stitch = stitcher.stitch_images(H, idx_ref, T_norm, idxs , beautify=beautify, size = warp_shape) #Stitch all the images
    
    if save_output:
        cv.imwrite(os.path.join(output_dir,"stitched.jpg"), cv.cvtColor(stitch,cv.COLOR_RGB2BGR))
    
    if verbose:
        figure(figsize=(40, 40), dpi=80)
        plt.imshow(stitch,),plt.show() 

    return H, stitch